# Borrower Reliability Study

This research is relevant to the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.

The results of the study could be taken into account when building a credit scoring model - a special system that assesses the ability of a potential borrower to repay a loan to a bank.

## Importing and familiarization with the data

In [1]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2022-06-22 19:04:21--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.245, 5.45.205.241, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.245|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cachev2-spb02.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=122 [following]
--2022-06-22 19:04:21--  https://cachev2-spb02.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=122
Resolving cachev2-spb02.cdn.yandex.net (cachev2-spb02.cdn.yandex.net)... 37.140.137.4, 2a02:6b8:0:2221::304
Connecting to cachev2-spb02.cdn.yandex.net (cachev2-spb02.cdn.yandex.net)|37.140.137.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[

In [2]:
from pymystem3 import Mystem
m = Mystem()

In [3]:
import pandas as pd
data = pd.read_csv('/content/project-2-data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
data[data['total_income'].isna()]['days_employed'].isna().count()

2174

Gaps are present in both columns at the same time.

Let's check the content of the columns with categorical data and fix the detected problems:

In [6]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [7]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [8]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [9]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [10]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [11]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [12]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [13]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [14]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [15]:
data[data['dob_years'] == 0]['dob_years'].count()

101

In [16]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [17]:
data[data['children'] == -1]['children'].count()

47

In [18]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

In [19]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [20]:
data['debt'].unique()

array([0, 1])

##Data preprocessing

### Processing of missing values

In [21]:
data[data['total_income'].isna()]['income_type'].value_counts(normalize=True)

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

In [22]:
data[~data['total_income'].isna()]['income_type'].value_counts(normalize=True)

сотрудник          0.517493
компаньон          0.236525
пенсионер          0.177924
госслужащий        0.067800
безработный        0.000103
студент            0.000052
предприниматель    0.000052
в декрете          0.000052
Name: income_type, dtype: float64

In [23]:
data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [24]:
data[data['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [25]:
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

In [26]:
data = data[data['children'] != -1]
data['days_employed'] = data['days_employed'].fillna(0)
data.isna().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

We got rid of lines with an unspecified number of children. Gaps in the length of service and income are replaced by zeros for the possibility of further transformation of the data type in these columns.

### Changing the data type

In [27]:
data['days_employed'] = data['days_employed'].astype('int').abs()
data['total_income'] = data['total_income'].astype('int').abs()
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [28]:
data.loc[(data['dob_years'] - data['days_employed'] / 365) < 0]['days_employed'].count()

3512

We brought the columns with data on experience and income to a digestible form. It is noteworthy: it was found that the experience in 3512 cases is greater than the age of the borrowers. Thus, another error in the data was discovered. But the experience in our case can be neglected, because it is not within the scope of the study.

### Processing duplicated entries

In [29]:
data.duplicated().sum()

71

In [30]:
data = data.drop_duplicates().reset_index(drop=True)

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21407 entries, 0 to 21406
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21407 non-null  int64 
 1   days_employed     21407 non-null  int64 
 2   dob_years         21407 non-null  int64 
 3   education         21407 non-null  object
 4   education_id      21407 non-null  int64 
 5   family_status     21407 non-null  object
 6   family_status_id  21407 non-null  int64 
 7   gender            21407 non-null  object
 8   income_type       21407 non-null  object
 9   debt              21407 non-null  int64 
 10  total_income      21407 non-null  int64 
 11  purpose           21407 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Lemmatization

In [32]:
def credit_purpose(purpose):
    '''The function categorizes the purpose of obtaining a loan 
    based on the lemmatization of the values of the "purpose" column'''
    lemm_purpose = m.lemmatize(purpose)
    if 'коммерческий' in lemm_purpose:
        return 'коммерческая недвижимость'
    if 'жилье' in lemm_purpose:
        return 'жилая недвижимость'
    if 'недвижимость' in lemm_purpose:
        return 'жилая недвижимость'
    if 'автомобиль' in lemm_purpose:
        return 'автомобиль'
    if 'свадьба' in lemm_purpose:
        return 'свадьба'
    if 'образование' in lemm_purpose:
        return 'образование'

In [33]:
data['purpose_category'] = data['purpose'].apply(credit_purpose)

In [34]:
data.groupby('purpose_category')['purpose_category'].count()

purpose_category
автомобиль                   4295
жилая недвижимость           9477
коммерческая недвижимость    1310
образование                  4003
свадьба                      2322
Name: purpose_category, dtype: int64

In [35]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилая недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилая недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


The function of lemmatization of the values of the column of goals for obtaining a loan has been created for further categorization of data by this attribute. A column with categories of purposes for obtaining a loan has been added to the table.

### Data categorization

Breakdown by categories of purpose of the loan:

In [36]:
data['purpose_category'].value_counts()

жилая недвижимость           9477
автомобиль                   4295
образование                  4003
свадьба                      2322
коммерческая недвижимость    1310
Name: purpose_category, dtype: int64

Because income values is a numeric attribute, it is necessary to enter income level categories for further analysis.

Let's look at the distribution of the values in the income column to determine the categorization intervals:

In [37]:
data[data['total_income'] != 0]['total_income'].describe()

count    2.140700e+04
mean     1.653460e+05
std      9.824362e+04
min      2.066700e+04
25%      1.075550e+05
50%      1.425940e+05
75%      1.958685e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [38]:
data['income_category'] = pd.qcut(data['total_income'], 6)

In [39]:
data.groupby('income_category')['total_income'].count()

income_category
(20666.999, 92116.333]     3568
(92116.333, 119257.333]    3568
(119257.333, 142594.0]     3684
(142594.0, 172357.0]       3580
(172357.0, 228922.333]     3439
(228922.333, 2265604.0]    3568
Name: total_income, dtype: int64

The categorization of data was carried out according to the purpose of obtaining a loan and the level of income of the borrower.

## Answering questions

- Is there a relationship between having children and repaying a loan on time?

In [40]:
children_data = data.groupby('children')['debt'].agg(['count', 'mean'])
make_percent = lambda x: "{:,.2%}".format(x)
children_data['mean'] = children_data['mean'].apply(make_percent)
children_data

,count,mean
children,,
0,14091,7.54%
1,4808,9.23%
2,2052,9.45%
3,330,8.18%
4,41,9.76%
5,9,0.00%
20,76,10.53%


The data shows that the presence of children increases the share of non-performing loans from 7.5% to 8-10%, depending on the number of children. However, the amount of data for cases with four or more children is too small to be representative.

- Is there a relationship between marital status and loan repayment on time?

In [41]:
family_status_data = (data.groupby('family_status')['debt'].
                      agg(['count', 'mean']).
                      sort_values(by='mean', ascending=False))
family_status_data['mean'] = family_status_data['mean'].map('{:,.2%}'.format)
family_status_data

,count,mean
family_status,,
не женат / не замужем,2805,9.77%
гражданский брак,4146,9.36%
женат / замужем,12310,7.55%
в разводе,1191,7.14%
вдовец / вдова,955,6.60%


Conduct an additional check of the age distribution of borrowers in the categories of marital status:

In [42]:
(data.groupby('family_status')['debt', 'dob_years'].
 agg(['mean']).sort_values(by=('debt','mean'), ascending=False))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,debt,dob_years
,mean,mean
family_status,,
не женат / не замужем,0.097683,38.346168
гражданский брак,0.093584,42.078389
женат / замужем,0.075548,43.548985
в разводе,0.071369,45.555835
вдовец / вдова,0.065969,56.520419


The share of non-performing loans decreases along with the decrease in the "degree of freedom" of the borrower. In this case, there is also a dependence on the age of borrowers in these categories.

- Is there a relationship between income level and loan repayment on time?

In [43]:
income_category_data = data.groupby('income_category')['debt'].agg(['count', 'mean'])
income_category_data['mean'] = income_category_data['mean'].map('{:,.2%}'.format)
income_category_data

,count,mean
income_category,,
"(20666.999, 92116.333]",3568,7.99%
"(92116.333, 119257.333]",3568,8.30%
"(119257.333, 142594.0]",3684,8.93%
"(142594.0, 172357.0]",3580,8.44%
"(172357.0, 228922.333]",3439,8.08%
"(228922.333, 2265604.0]",3568,7.01%


A larger number of debts is shown by borrowers with an average income level, relative to the sample. With the achievement of an income level of more than 173 thousand, the share of non-performing loans begins to decline. Low-income borrowers (Group 1) demonstrate a more responsible attitude towards loan repayment.

- How do different purposes of a loan affect its repayment on time?

In [44]:
purpose_data = (data.groupby('purpose_category')['debt'].
                agg(['count', 'mean']).sort_values(by='mean', ascending=False))
purpose_data['mean'] = purpose_data['mean'].map('{:,.2%}'.format)
purpose_data

,count,mean
purpose_category,,
автомобиль,4295,9.36%
образование,4003,9.24%
свадьба,2322,8.01%
коммерческая недвижимость,1310,7.56%
жилая недвижимость,9477,7.21%


The purpose of the loan has a significant impact on the percentage of debts. The most responsible borrowers are real estate buyers, and the purpose of the property is not significant. Buying a car and getting an education are the most risky categories in terms of timely loan repayment.

## Final conclusion

A study was made of the influence of various factors on the share of loans not returned on time. Loading, preprocessing and categorization of data was performed in accordance with the requirements of the task. Answers to the questions posed were received and conclusions were formulated regarding each subsection of the study. The study showed the presence of dependencies in all situations under consideration.